In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_score, recall_score, f1_score,
    confusion_matrix)
from xgboost import XGBClassifier

df = pd.read_csv("d:data/startup/hospital_readmission_model_ready.csv")
target = "readmit_30"

print("Readmit Rate", df[target].mean())

df.head(2)

C:\Users\syeds\AppData\Local\Temp\ipykernel_2628\3371395273.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("d:data/startup/hospital_readmission_model_ready.csv")


Readmit Rate 0.11159915885462728


,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,diabetesMed,readmitted,readmit_30,discharge_disposition,admission_type,admission_source,los_bucket,prior_util_total,meds_bucket_clean,high_meds_flag
0,Caucasian,Female,[0-10),6,25,1,1,NaN,Pediatrics-Endocrinology,41,...,No,NO,0,Not Mapped,Unknown,Physician Referral,0-2,0,1-9,0
1,Caucasian,Female,[10-20),1,1,7,3,NaN,NaN,59,...,Yes,>30,0,Discharged to home,Emergency,Emergency Room,3-5,0,18-22,0


In [8]:
#train -test split
target = "readmit_30"

X = df.drop(columns=[target, "readmitted"], errors="ignore")  # <- important
y = df[target].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=102, stratify=y)
print("Training data readmit rate", y_train.mean())
print("Testing data readmit rate", y_test.mean())

Training data readmit rate 0.11160516877118852
Testing data readmit rate 0.11157512036946055


# preprocessing and modeling

In [9]:
# column types
num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

preprocess = ColumnTransformer(
    transformers =[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"),cat_cols),
    ]  
)

logreg = Pipeline(
    steps =[
        ("prep", preprocess),
        ("model", LogisticRegression(max_iter=2000, class_weight= "balanced"))
    ]
)
logreg

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['admission_type_id',
                                                   'discharge_disposition_id',
                                                   'admission_source_id',
                                                   'time_in_hospital',
                                                   'num_lab_procedures',
                                                   'num_procedures',
                                                   'num_medications',
                                                   'number_outpatient',
                                                   'number_emergency',
                                                   'number_inpatient',
                                                   'number_diagnoses',
                                                   'prior_util_total',
                                                   'high_meds_flag']),
                                                 ('cat',
                                                  One...
                                                   'acetohexamide', 'glipizide',
                                                   'glyburide', 'tolbutamide',
                                                   'pioglitazone',
                                                   'rosiglitazone', 'acarbose',
                                                   'miglitol', 'troglitazone',
                                                   'tolazamide', 'examide',
                                                   'citoglipton', 'insulin',
                                                   'glyburide-metformin',
                                                   'glipizide-metformin',
                                                   'glimepiride-pioglitazone',
                                                   'metformin-rosiglitazone',
                                                   'metformin-pioglitazone', ...])])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=2000))])

# Evaluation

In [10]:
def model_eval(model, X_test, y_test, threshold =0.5, name = "model"):
    probability =model.predict_proba(X_test)[:,1]
    prediction = (probability >= threshold).astype(int)
    
    roc = roc_auc_score(y_test, probability)
    pr = average_precision_score(y_test, probability)
    precision = precision_score(y_test, prediction, zero_division=0)
    recall = recall_score(y_test, prediction, zero_division=0)
    f1 = f1_score(y_test, prediction, zero_division=0)
    cm = confusion_matrix(y_test, prediction)
    
    print(f"\n{name} @ threshold={threshold}")
    print(f"ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    print("Confusion matrix:\n", cm)
    
    return probability

In [11]:
logreg.fit(X_train, y_train)
logreg_proba = model_eval(logreg, X_test, y_test, threshold=0.5, name="LogReg (balanced)")



LogReg (balanced) @ threshold=0.5
ROC-AUC: 0.6796 | PR-AUC: 0.2257 | Precision: 0.1854 | Recall: 0.5782 | F1: 0.2808
Confusion matrix:
 [[12314  5769]
 [  958  1313]]


### Baseline logistic regression achieves ROC-AUC ~0.68; at 0.5 threshold it captures ~58% of readmissions (recall) with ~19% precision, consistent with a screening-style risk model.

In [12]:
#testing wth 'n' numbers of threshold to test recall rate
probability = logreg.predict_proba(X_test)[:, 1]

print("t   | flagged% | precision | recall | f1")
for t in np.arange(0.05, 0.75, 0.05):
    preds = (probability >= t).astype(int)
    flagged = preds.mean()  # fraction of encounters flagged high-risk
    prec = precision_score(y_test, preds, zero_division=0)
    rec  = recall_score(y_test, preds, zero_division=0)
    f1   = f1_score(y_test, preds, zero_division=0)
    print(f"{t:0.2f} |  {flagged:0.3f}   |   {prec:0.3f}   |  {rec:0.3f} | {f1:0.3f}")

t   | flagged% | precision | recall | f1
0.05 |  0.983   |   0.114   |  1.000 | 0.204
0.10 |  0.981   |   0.114   |  1.000 | 0.204
0.15 |  0.979   |   0.114   |  0.998 | 0.204
0.20 |  0.973   |   0.114   |  0.997 | 0.205
0.25 |  0.961   |   0.115   |  0.992 | 0.206
0.30 |  0.923   |   0.118   |  0.976 | 0.210
0.35 |  0.822   |   0.127   |  0.932 | 0.223
0.40 |  0.666   |   0.141   |  0.841 | 0.242
0.45 |  0.494   |   0.163   |  0.721 | 0.266
0.50 |  0.348   |   0.185   |  0.578 | 0.281
0.55 |  0.232   |   0.211   |  0.438 | 0.284
0.60 |  0.154   |   0.238   |  0.329 | 0.277
0.65 |  0.099   |   0.266   |  0.236 | 0.250
0.70 |  0.065   |   0.298   |  0.173 | 0.219


In [13]:
#XGBoost

xgb = Pipeline(
    steps=[
        ("prep", preprocess),
        ("model", XGBClassifier(
            n_estimators=400,
            max_depth=4,
            learning_rate=0.05,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            random_state=42,
            eval_metric="logloss"
        ))
    ]
)

xgb.fit(X_train, y_train)
xgb_proba = model_eval(xgb, X_test, y_test, threshold=0.5, name="XGBoost")



XGBoost @ threshold=0.5
ROC-AUC: 0.6877 | PR-AUC: 0.2436 | Precision: 0.6512 | Recall: 0.0123 | F1: 0.0242
Confusion matrix:
 [[18068    15]
 [ 2243    28]]


In [14]:
#Best model
import joblib

logreg_proba = logreg.predict_proba(X_test)[:, 1]
logreg_prauc = average_precision_score(y_test, logreg_proba)

best_model = logreg
best_name = "LogReg"
best_prauc = logreg_prauc

# If xgb exists, compare
if "xgb" in globals():
    xgb_proba = xgb.predict_proba(X_test)[:, 1]
    xgb_prauc = average_precision_score(y_test, xgb_proba)
    if xgb_prauc > best_prauc:
        best_model = xgb
        best_name = "XGBoost"
        best_prauc = xgb_prauc

MODEL_OUT = r"D:\data\startup\best_readmission_model.pkl"
joblib.dump(best_model, MODEL_OUT)

print("Best model:", best_name)
print("Best PR-AUC:", best_prauc)
print("Saved:", MODEL_OUT)


Best model: XGBoost
Best PR-AUC: 0.24359380109184264
Saved: D:\data\startup\best_readmission_model.pkl


In [15]:
# Probabilities from best model (XGBoost)
proba = best_model.predict_proba(X_test)[:, 1]

risk = X_test.copy()
risk["readmit_prob"] = proba
risk["readmit_30_actual"] = y_test.values

# assigning tiers for high and med risk
t_high = 0.60
t_med  = 0.55

risk["risk_tier"] = np.where(
    risk["readmit_prob"] >= t_high, "High",
    np.where(risk["readmit_prob"] >= t_med, "Medium", "Low")
)

# Tier distribution
print("Tier distribution:\n", risk["risk_tier"].value_counts(normalize=True))

# Evaluating "High+Medium" as screening set
pred_screen = (risk["readmit_prob"] >= t_med).astype(int)
print("\nScreening set (High+Medium):")
print("Flagged%:", pred_screen.mean())
print("Precision:", precision_score(y_test, pred_screen, zero_division=0))
print("Recall:", recall_score(y_test, pred_screen, zero_division=0))

OUT_RISK = r"D:\data\startup\pbi_patient_risk_scores.csv"
risk.to_csv(OUT_RISK, index=False)
print("\nSaved:", OUT_RISK)


Tier distribution:
 risk_tier
Low       0.998919
Medium    0.000737
High      0.000344
Name: proportion, dtype: float64

Screening set (High+Medium):
Flagged%: 0.0010808686253316302
Precision: 0.7272727272727273
Recall: 0.007045354469396741

Saved: D:\data\startup\pbi_patient_risk_scores.csv


### We selected thresholds based on intervention capacity. Flagging ~23% of encounters (High+Medium risk) captures ~42% of all 30-day readmissions with ~21% precision, providing a manageable follow-up list while concentrating readmission risk.”

High risk = “most likely to come back”

Medium risk = “somewhat likely”

Low risk = “less likely”

Our results say:

77% Low risk → most people are probably fine

15% High risk → these are the ones you should worry about most

7% Medium risk → some risk, worth attention if you have capacity

So basically, out of 100 visits, our model says ~15 are high risk, ~7 medium, ~77 low.



In [16]:
print(df.shape)
print("model_ready cols:", pd.read_csv(r"D:\data\startup\hospital_readmission_model_ready.csv").shape)
print("risk file cols:", pd.read_csv(r"D:\data\startup\pbi_patient_risk_scores.csv").shape)


(101766, 52)


C:\Users\syeds\AppData\Local\Temp\ipykernel_2628\3183961771.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  print("model_ready cols:", pd.read_csv(r"D:\data\startup\hospital_readmission_model_ready.csv").shape)


model_ready cols: (101766, 52)
risk file cols: (20354, 53)


In [17]:
#risk tiers for entire ds
X_all = df.drop(columns=["readmit_30", "readmitted"], errors="ignore")
proba_all = best_model.predict_proba(X_all)[:, 1]

risk_all = X_all.copy()
risk_all["readmit_prob"] = proba_all
risk_all["readmit_30_actual"] = df["readmit_30"].values

t_high = 0.60
t_med  = 0.55

risk_all["risk_tier"] = np.where(
    risk_all["readmit_prob"] >= t_high, "High",
    np.where(risk_all["readmit_prob"] >= t_med, "Medium", "Low")
)

OUT_ALL = r"D:\data\startup\pbi_patient_risk_scores_ALL.csv"
risk_all.to_csv(OUT_ALL, index=False)
print("Saved:", OUT_ALL, risk_all.shape)


Saved: D:\data\startup\pbi_patient_risk_scores_ALL.csv (101766, 53)
